In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#! conda install -c conda-forge shapely --yes
#from shapely.geometry import MultiPoint

#! conda install -c conda-forge geopandas --yes
#import geopandas as gpd

#!conda update conda --yes
#!conda update conda-build --yes
print('Libraries imported.')

Libraries imported.


In [7]:
from geopy.geocoders import Nominatim
address = 'Salerno'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Salerno are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Salerno are 40.6803601, 14.7594542.


In [39]:
coordinates=pd.read_csv(r'CAP_SALERNO.csv', sep=";")
coordinates

,CAP,Latitude,Longitude,Unnamed: 3
0,84121,40.677973,14.749906,NaN
1,84122,40.677126,14.765950,NaN
2,84123,40.676365,14.771784,NaN
3,84124,40.681521,14.770326,NaN
4,84125,40.692343,14.749177,NaN
5,84126,40.685770,14.771055,NaN
6,84127,40.671500,14.780536,NaN
7,84128,40.667875,14.789289,NaN
8,84129,40.662482,14.797312,NaN
9,84130,40.682441,14.768096,NaN


In [40]:
coordinates['Latitude']=coordinates['Latitude'].astype(float)
coordinates['Longitude']=coordinates['Longitude'].astype(float)
coordinates['CAP']=coordinates['CAP'].astype(str)

## Insert Foursquare credential

In [8]:
CLIENT_ID = 'EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ' # your Foursquare ID
CLIENT_SECRET = 'BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ
CLIENT_SECRET:BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX


In [9]:
Borough_latitude = coordinates.loc[0, 'Latitude'] # Borough latitude value
Borough_longitude = coordinates.loc[0, 'Longitude'] # Borough longitude value

Borough_name = coordinates.loc[0, 'CAP'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(Borough_name, 
                                                               Borough_latitude, 
                                                               Borough_longitude))

Latitude and longitude values of 84121 are 40.677973, 14.749906.


In [10]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Borough_latitude, 
    Borough_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=EKDAJ1PSVFYIVBM0LVHQ04SU2ZNCVDWTU4RUH04EY1OQFIPQ&client_secret=BKFIJAUYK1ZRYGR543ZBBJAT2Z0RTAZOOMU3KGJC3AR3O4XX&v=20180605&ll=40.677973,14.749906&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1f4e3abe707800320eaae5'},
 'response': {'headerLocation': 'Salerno',
  'headerFullLocation': 'Salerno',
  'headerLocationGranularity': 'city',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 40.682473004500004,
    'lng': 14.755828588953015},
   'sw': {'lat': 40.6734729955, 'lng': 14.743983411046983}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c758c993adda143d0aa07af',
       'name': 'Teatro Verdi',
       'location': {'address': 'Via Roma, 254-260',
        'lat': 40.679121309680745,
        'lng': 14.751931574179546,
        'labeledLatLngs': [{'label': 'display',
          'lat': 40.679121309680745,
          'lng': 14.751931574179546}],
        'distance': 213,
        'postalCode': '84121',
        'cc': 'IT',


In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Teatro Verdi,Theater,40.679121,14.751932
1,Villa Comunale,Garden,40.678409,14.753501
2,Resilienza,Pizza Place,40.678309,14.750759
3,Spiaggia Santa Teresa,Beach,40.677569,14.755690
4,Portovecchio,Seafood Restaurant,40.677350,14.751810
5,Bar Verdi,Bar,40.679419,14.752643
6,Galleon Pub,Italian Restaurant,40.679129,14.755055
7,Giardino della Minerva,Historic Site,40.680906,14.753962
8,Bar Umberto,Bar,40.678935,14.755272
9,Hotel VillaPoseidon MaisonDeCharme,Hotel Bar,40.680197,14.749681


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Salerno_venues = getNearbyVenues(names=coordinates['CAP'],
                                   latitudes=coordinates['Latitude'],
                                   longitudes=coordinates['Longitude']
                                  )


84121
84122
84123
84124
84125
84126
84127
84128
84129
84130
84131
84132
84133
84134
84135


In [18]:
print(Salerno_venues.shape)
Salerno_venues

(179, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,84121,40.677973,14.749906,Teatro Verdi,40.679121,14.751932,Theater
1,84121,40.677973,14.749906,Villa Comunale,40.678409,14.753501,Garden
2,84121,40.677973,14.749906,Resilienza,40.678309,14.750759,Pizza Place
3,84121,40.677973,14.749906,Spiaggia Santa Teresa,40.677569,14.755690,Beach
4,84121,40.677973,14.749906,Portovecchio,40.677350,14.751810,Seafood Restaurant
5,84121,40.677973,14.749906,Bar Verdi,40.679419,14.752643,Bar
6,84121,40.677973,14.749906,Galleon Pub,40.679129,14.755055,Italian Restaurant
7,84121,40.677973,14.749906,Giardino della Minerva,40.680906,14.753962,Historic Site
8,84121,40.677973,14.749906,Bar Umberto,40.678935,14.755272,Bar
9,84121,40.677973,14.749906,Hotel VillaPoseidon MaisonDeCharme,40.680197,14.749681,Hotel Bar


In [19]:
DB_SA_PP_foursquare=Salerno_venues.loc[Salerno_venues['Venue Category'] == 'Pizza Place']
DB_SA_PP_foursquare

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,84121,40.677973,14.749906,Resilienza,40.678309,14.750759,Pizza Place
15,84121,40.677973,14.749906,Totó Sapore,40.678915,14.755355,Pizza Place
44,84122,40.677126,14.765950,Pizzeria Trianon,40.677456,14.763068,Pizza Place
47,84122,40.677126,14.765950,Anema e Cozze,40.675857,14.765339,Pizza Place
50,84122,40.677126,14.765950,Pizza Margherita,40.676251,14.765383,Pizza Place
59,84122,40.677126,14.765950,Addor i Pizza,40.680320,14.767337,Pizza Place
78,84123,40.676365,14.771784,Pizzeria Sorbillo,40.674082,14.772499,Pizza Place
84,84123,40.676365,14.771784,Pizzicorio,40.673950,14.770239,Pizza Place
105,84123,40.676365,14.771784,Ristorante Pizzeria La Bodeguita,40.677475,14.776015,Pizza Place
106,84123,40.676365,14.771784,Ristorante Pizzeria Il Graal,40.676891,14.776275,Pizza Place


In [20]:
DB_SA_PP_foursquare.shape[0]

16

In [21]:
DB_SA_PP_GoogleMaps= pd.read_csv(r'pizzerie.csv',sep=';')

In [22]:
DB_SA_PP_GoogleMaps

,Name,CAP,Rating,Full_Address,latitude,longitude
0,Pizzamarè Ristorante,84100,4.7,Via Ligea 110/112,40.677509,14.743154
1,GALLEON Pub,84100,3.8,Via Roma 256,40.679148,14.754955
2,Pizzeria Ristorante Mare Terra e....Pizza,84100,3.9,Via Leucosia 9,40.657182,14.802590
3,Pizzeria Salernitana,84100,4.1,Via Salvatore Allende,40.647147,14.816318
4,Pizzeria Reginè,84100,4.3,Via Posidonia 139,40.669073,14.787172
5,Vicolo della Neve,84100,4.1,Vicolo della Neve 24,40.679100,14.758050
6,Pizzeria I Due Fratelli,84100,4.4,Via Martiri Ungheresi 10,40.665029,14.794388
7,Capri,84100,4.1,Via Torrione 36,40.672932,14.776428
8,Complesso Turistico Centro Storico Salerno Gen...,84121,NaN,Via Roma 202,40.679022,14.756189
9,Salerno Vecchia Pizzeria di Marco Gabini,84121,4.0,centro SA IT Piazza Sedile di Portarotese 16,40.681182,14.760964


In [23]:
DB_SA_PP_GoogleMaps.shape[0]

153

In [24]:
DB_SA_PP_GoogleMaps['latitude']=DB_SA_PP_GoogleMaps['latitude'].astype(float)
DB_SA_PP_GoogleMaps['longituìtude']=DB_SA_PP_GoogleMaps['longitude'].astype(float)
DB_SA_PP_GoogleMaps['Name']=DB_SA_PP_GoogleMaps['Name'].astype(str)

In [25]:
# create map of Salerno using latitude and longitude values
map_Salerno_PP = folium.Map(location=[latitude, longitude], zoom_start=10,control_scale = True)
i=0
j=0
# add markers to map
for  i in range(0,len(DB_SA_PP_foursquare)):
    folium.CircleMarker([DB_SA_PP_foursquare.iloc[i]['Venue Latitude'], DB_SA_PP_foursquare.iloc[i]['Venue Longitude']],
                        #popup=DB_SA_PP_foursquare.iloc[i]['Venue'],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        #parse_html=True
                       ).add_to(map_Salerno_PP)
for  j in range(0,len(DB_SA_PP_GoogleMaps)):
    folium.CircleMarker([DB_SA_PP_GoogleMaps.iloc[j]['latitude'], DB_SA_PP_GoogleMaps.iloc[j]['longitude']],
                        #popup=DB_SA_PP_GoogleMaps.iloc[j]['Name'],
                        radius=2,
                        color='red',
                        fill=False,
                        fill_color='red',
                        fill_opacity=0.7,
                        #parse_html=True
                       ).add_to(map_Salerno_PP)
    
map_Salerno_PP

In [26]:
import pandas as pd
DB_SA_PP= pd.read_csv(r'pizzerie_UNION.csv',sep=';')

In [27]:
DB_SA_PP.head()

,Name,CAP,Rating,Full_Address,latitude,longitude,Unnamed: 6
0,Pizzamarè Ristorante,84100,4.7,Via Ligea 110/112,40.677509,14.743154,NaN
1,GALLEON Pub,84100,3.8,Via Roma 256,40.679148,14.754955,NaN
2,Pizzeria Ristorante Mare Terra e....Pizza,84100,3.9,Via Leucosia 9,40.657182,14.802590,NaN
3,Pizzeria Salernitana,84100,4.1,Via Salvatore Allende,40.647147,14.816318,NaN
4,Pizzeria Reginè,84100,4.3,Via Posidonia 139,40.669073,14.787172,NaN


In [28]:
# create map of Salerno using latitude and longitude values
map_Salerno_PP = folium.Map(location=[latitude, longitude], zoom_start=10,control_scale = True)
i=0
j=0
# add markers to map
for  i in range(0,len(DB_SA_PP)):
    folium.CircleMarker([DB_SA_PP.iloc[i]['latitude'], DB_SA_PP.iloc[i]['longitude']],
                        #popup=DB_SA_PP_foursquare.iloc[i]['Venue'],
                        radius=2,
                        color='black',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        #parse_html=True
                       ).add_to(map_Salerno_PP)

    
map_Salerno_PP

In [35]:
df2=DB_SA_PP.drop({'Name','CAP','Full_Address','Unnamed: 6','Rating'}, axis=1)
from sklearn.cluster import DBSCAN
db2 = DBSCAN(eps=0.0030, min_samples=2).fit(df2)
core_samples_mask = np.zeros_like(db2.labels_, dtype=bool)
core_samples_mask[db2.core_sample_indices_] = True
labels = db2.labels_
df2["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 


# A sample of clusters
df2

,latitude,longitude,Clus_Db
0,40.677509,14.743154,0
1,40.679148,14.754955,1
2,40.657182,14.802590,2
3,40.647147,14.816318,-1
4,40.669073,14.787172,3
5,40.679100,14.758050,1
6,40.665029,14.794388,3
7,40.672932,14.776428,4
8,40.679022,14.756189,1
9,40.681182,14.760964,1


In [36]:
import folium 
map_Salerno_PP = folium.Map(location=[latitude, longitude], zoom_start=10,control_scale = True)
colordict={-1: 'red', 0: 'yellow', 1: 'darkblue', 2: 'orange', 3: 'darkpurple', 4: 'pink',5: 'green',
           6: 'blue',7:'gray',8:'darkred',9:'darkgreen'}
for lat, lon, Clus in zip(df2['latitude'], df2['longitude'], df2['Clus_Db']):
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        color='none',
        key_on = Clus,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[Clus],
        fill=True,
        fill_opacity=0.7
        ).add_to(map_Salerno_PP)   
map_Salerno_PP